In [14]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
import h5py
from stellarperceptron.model import StellarPerceptron
import numpy as np
from datetime import datetime
# !python training.py

True
0


In [41]:
with h5py.File("./data_files/training_set.h5", mode="r") as xp_apogee:
    print(xp_apogee.keys())
    print(xp_apogee['raw'])
    print(xp_apogee['raw'].keys())
    xp_relevancy = xp_apogee['raw']['xp_relevancy'][()]
    xp_coeffs_gnorm = xp_apogee['raw']['xp_coeffs_gnorm'][()]
    xp_coeffs_err_gnorm = xp_apogee['raw']['xp_coeffs_gnorm'][()]
    # print(xp_relevancy.shape)
    # print(xp_coeffs_gnorm.shape)
    # print(xp_coeffs_err_gnorm.shape)
    # print("---"*20)
    # print(xp_relevancy[:10,51:52])
    # print(xp_relevancy[:10,52:55])
    # print("---"*20)
    # print(~xp_relevancy[:10,51:52])
    # print(~xp_relevancy[:10,52:55])
    # print(xp_relevancy[:30,106:107])
    # print(xp_relevancy[:30,107:110])
    # print("---"*20)
    # print(xp_coeffs_gnorm[:30,51:52])
    # print(xp_coeffs_gnorm[:30,52:55])
    # print(xp_coeffs_gnorm[:30,106:107])
    # print(xp_coeffs_gnorm[:30,107:110])
    # print("---"*20)
    # print(xp_coeffs_err_gnorm[:30,51:52])
    # print(xp_coeffs_err_gnorm[:30,52:55])
    # print(xp_coeffs_err_gnorm[:30,106:107])
    # print(xp_coeffs_err_gnorm[:30,107:110])
    xp_relevancy[:,52:55] = xp_relevancy[:,51:52]
    xp_relevancy[:,107:110] = xp_relevancy[:,106:107]
    # print(xp_coeffs_gnorm[~xp_relevancy][:200])
    # print(xp_coeffs_err_gnorm[~xp_relevancy][:200])
    xp_coeffs_gnorm[~xp_relevancy] = np.nan
    xp_coeffs_err_gnorm[~xp_relevancy] = np.nan
    
    print("---"*20)
    print(xp_coeffs_gnorm.shape)
    print(xp_apogee['raw']['bprp'].shape)
    print(xp_apogee['raw']['jh'].shape)
    print(xp_apogee['raw']['jk'].shape)
    print(xp_apogee['raw']['teff'].shape)
    print(xp_apogee['raw']['logg'].shape)
    print(xp_apogee['raw']['m_h'].shape)
    print(xp_apogee['raw']['logc19'].shape)
    print(xp_apogee['raw']['g_fakemag'].shape)
    
    print("---"*20)
    training_labels = np.column_stack(
        [
            xp_coeffs_gnorm,
            xp_apogee['raw']['bprp'][()],
            xp_apogee['raw']['jh'][()],
            xp_apogee['raw']['jk'][()],
            xp_apogee['raw']['teff'][()],
            xp_apogee['raw']['logg'][()],
            xp_apogee['raw']['m_h'][()],
            xp_apogee['raw']['logc19'][()],
            xp_apogee['raw']['g_fakemag'][()],
        ]
    )
    print(training_labels.shape)
    training_labels_err = np.column_stack(
        [
            xp_coeffs_gnorm,
            xp_apogee['raw']['bprp_err'][()],
            xp_apogee['raw']['jh_err'][()],
            xp_apogee['raw']['jk_err'][()],
            xp_apogee['raw']['teff_err'][()],
            xp_apogee['raw']['logg_err'][()],
            xp_apogee['raw']['m_h_err'][()],
            xp_apogee['raw']['logc19_err'][()],
            xp_apogee['raw']['g_fakemag_err'][()],
        ]
    )
    print(training_labels_err.shape)
    # print('-'*100)
    # print(training_labels_err[:10,:3])
    # print(np.isnan(training_labels_err)[:,0].sum())
    training_labels_err = np.where(np.isnan(training_labels_err), 0.0, training_labels_err)
    # print(training_labels_err[:10,:3])
    # print(np.isnan(training_labels_err).sum())

<KeysViewHDF5 ['2mass_designation', 'gaia_source_id', 'raw']>
<HDF5 group "/raw" (24 members)>
<KeysViewHDF5 ['bprp', 'bprp_err', 'g_fakemag', 'g_fakemag_err', 'jh', 'jh_err', 'jk', 'jk_err', 'logc19', 'logc19_err', 'logg', 'logg_err', 'm_h', 'm_h_err', 'parallax', 'parallax_err', 'phot_g_mean_mag', 'teff', 'teff_err', 'xp_coeffs', 'xp_coeffs_err', 'xp_coeffs_gnorm', 'xp_coeffs_gnorm_err', 'xp_relevancy']>
------------------------------------------------------------
(396718, 110)
(396718,)
(396718,)
(396718,)
(396718,)
(396718,)
(396718,)
(396718,)
(396718,)
------------------------------------------------------------
(396718, 118)
(396718, 118)


In [42]:
# print([f"bp{i}" for i in range(55)])
# print(*[f"bp{i}" for i in range(55)])
obs_names = [
    *[f"bp{i}" for i in range(55)],
    *[f"rp{i}" for i in range(55)],
    "bprp",
    "jh",
    "jk",
    "teff",
    "logg",
    "m_h",
    "logebv",
    "g_fakemag",
]

In [43]:
print(obs_names)
print(*obs_names)

['bp0', 'bp1', 'bp2', 'bp3', 'bp4', 'bp5', 'bp6', 'bp7', 'bp8', 'bp9', 'bp10', 'bp11', 'bp12', 'bp13', 'bp14', 'bp15', 'bp16', 'bp17', 'bp18', 'bp19', 'bp20', 'bp21', 'bp22', 'bp23', 'bp24', 'bp25', 'bp26', 'bp27', 'bp28', 'bp29', 'bp30', 'bp31', 'bp32', 'bp33', 'bp34', 'bp35', 'bp36', 'bp37', 'bp38', 'bp39', 'bp40', 'bp41', 'bp42', 'bp43', 'bp44', 'bp45', 'bp46', 'bp47', 'bp48', 'bp49', 'bp50', 'bp51', 'bp52', 'bp53', 'bp54', 'rp0', 'rp1', 'rp2', 'rp3', 'rp4', 'rp5', 'rp6', 'rp7', 'rp8', 'rp9', 'rp10', 'rp11', 'rp12', 'rp13', 'rp14', 'rp15', 'rp16', 'rp17', 'rp18', 'rp19', 'rp20', 'rp21', 'rp22', 'rp23', 'rp24', 'rp25', 'rp26', 'rp27', 'rp28', 'rp29', 'rp30', 'rp31', 'rp32', 'rp33', 'rp34', 'rp35', 'rp36', 'rp37', 'rp38', 'rp39', 'rp40', 'rp41', 'rp42', 'rp43', 'rp44', 'rp45', 'rp46', 'rp47', 'rp48', 'rp49', 'rp50', 'rp51', 'rp52', 'rp53', 'rp54', 'bprp', 'jh', 'jk', 'teff', 'logg', 'm_h', 'logebv', 'g_fakemag']
bp0 bp1 bp2 bp3 bp4 bp5 bp6 bp7 bp8 bp9 bp10 bp11 bp12 bp13 bp14 bp15 bp1

In [45]:
now = datetime.now().strftime("%d%H%M%S")
# print(now)
device = 'cuda'
mixed_precision = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

context_length = 64
embedding_dim = 128

learning_rate = 1e-4
learning_rate_min = 1e-10
batch_size = 1024
epochs = 4096
cosine_annealing_t0 = 512
checkpoint_every_n_epochs = 128
save_model_to_folder = f"./model-{now}"
print(save_model_to_folder)

nn_model = StellarPerceptron(
    vocabs = obs_names,
    context_length = context_length,
    embedding_dim = embedding_dim,
    embedding_activation = "gelu",
    encoder_head_num = 16,
    encoder_dense_num = 1024,
    encoder_dropout_rate = 0.1,
    encoder_activation = "gelu",
    decoder_head_num = 16,
    decoder_dense_num = 3096,
    decoder_dropout_rate = 0.1,
    decoder_activation = "gelu",
    device = device,
    mixed_precision = mixed_precision,
    folder=save_model_to_folder,
)

nn_model.optimizer = torch.optim.AdamW(
    nn_model.torch_model.parameters(), lr=learning_rate
)

./model-28231628
